In [1]:
# pip install pyowm
from pyowm import OWM
from pyowm.utils import config
from pyowm.utils import timestamps
from datetime import datetime, timedelta, timezone
import pytz
import pandas as pd

API_KEY="4abf61c7e1edec294994f251711b2a92"

In [4]:
owm = OWM(API_KEY)
mgr = owm.weather_manager()


# Search for current weather in London (Great Britain) and get details
observation = mgr.weather_at_coords(lat=40.776676,lon=-73.971321)
w = observation.weather

# w.temperature('celsius') 
w.to_dict()


{'reference_time': 1657205603,
 'sunset_time': 1657240184,
 'sunrise_time': 1657186289,
 'clouds': 0,
 'rain': {},
 'snow': {},
 'wind': {'speed': 4.12, 'deg': 30},
 'humidity': 66,
 'pressure': {'press': 1016, 'sea_level': None},
 'temperature': {'temp': 298.17,
  'temp_kf': None,
  'temp_max': 300.57,
  'temp_min': 295.82,
  'feels_like': 298.45},
 'status': 'Clear',
 'detailed_status': 'clear sky',
 'weather_code': 800,
 'weather_icon_name': '01d',
 'visibility_distance': 10000,
 'dewpoint': None,
 'humidex': None,
 'heat_index': None,
 'utc_offset': -14400,
 'uvi': None,
 'precipitation_probability': None}

In [47]:
mgr = owm.weather_manager()

especifict_time = int(datetime(2018, 12, 1, 12).replace(tzinfo=pytz.timezone("US/Eastern")).timestamp())

one_call_especifict_time = mgr.one_call(lat=52.5244, lon=13.4105, dt=especifict_time)
one_call_especifict_time.current.to_dict()


{'reference_time': 1657214697,
 'sunset_time': 1657222179,
 'sunrise_time': 1657162346,
 'clouds': 75,
 'rain': {'1h': 4.21},
 'snow': {},
 'wind': {'speed': 4.12, 'deg': 280},
 'humidity': 87,
 'pressure': {'press': 1007, 'sea_level': None},
 'temperature': {'temp': 291.34, 'feels_like': 291.49},
 'status': 'Rain',
 'detailed_status': 'heavy intensity rain',
 'weather_code': 502,
 'weather_icon_name': '10d',
 'visibility_distance': 10000,
 'dewpoint': 289.14,
 'humidex': None,
 'heat_index': None,
 'utc_offset': None,
 'uvi': 0.47,
 'precipitation_probability': None}

In [46]:
one_call_especifict_time

<pyowm.weatherapi25.one_call.OneCall - lat=52.5244, lon=13.4105, retrieval_time=1657143884>

In [5]:
especifict_time

1656958020

- Manhattan, New York City, NY, USA (40.776676, -73.971321)
- Brooklyn, New York City, NY, USA (40.650002, -73.949997)
- Bronx, New York City, NY, USA (40.837048, -73.865433)
- Queens, New York City, NY, USA (40.742054, -73.769417)
- Staten Island, New York City, NY, USA (40.579021, -74.151535)

In [6]:
coords={
    "lat":{
        "EWR":40.6895314,
        "Manhattan":40.776676,
        "Brooklyn":40.650002,
        "Bronx":40.837048,
        "Queens":40.742054,
        "Staten Island":40.579021,
    },
    "lon":{
        "EWR": -74.17446239999998,
        "Manhattan":-73.971321,
        "Brooklyn": -73.949997,
        "Bronx":-73.865433,
        "Queens":-73.769417,
        "Staten Island":-74.151535,
    }
    
}

In [7]:
data=pd.read_parquet("../Data/yellow_tripdata_2018-01.parquet")

In [8]:
taxi_zone = pd.read_csv('../taxi+_zone_lookup.csv')
data_zone = data.merge(taxi_zone, left_on='PULocationID', right_on='LocationID')
data_zone.drop(['LocationID','service_zone'], axis=1, inplace=True)
data_zone.rename(columns={"Borough": "PUBorough", "Zone": "PUZone"}, inplace=True)

In [9]:
data_zone["PULat"]=data_zone["PUBorough"].map(coords["lat"])
data_zone["PULon"]=data_zone["PUBorough"].map(coords["lon"])



In [10]:
# data_zone_withoutNaNLatLon=data_zone[data_zone[["PULat"]].notna()]
data_zone_withoutNaNLatLon=data_zone.dropna(subset=['PULat'])
data_zone_withoutNaNLatLon.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,...,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee,PUBorough,PUZone,PULat,PULon
0,1,2018-01-01 00:21:05,2018-01-01 00:24:23,1,0.50,1,N,41,24,2,...,0.00,0.00,0.3,5.80,NaN,NaN,Manhattan,Central Harlem,40.776676,-73.971321
1,2,2018-01-01 00:49:32,2018-01-01 00:59:52,1,1.36,1,N,41,74,2,...,0.00,0.00,0.3,9.80,NaN,NaN,Manhattan,Central Harlem,40.776676,-73.971321
2,1,2018-01-01 00:11:56,2018-01-01 00:14:35,2,0.30,1,N,41,41,2,...,0.00,0.00,0.3,5.30,NaN,NaN,Manhattan,Central Harlem,40.776676,-73.971321
3,1,2018-01-01 00:58:32,2018-01-01 01:43:45,1,14.10,1,N,41,112,1,...,12.10,5.76,0.3,60.66,NaN,NaN,Manhattan,Central Harlem,40.776676,-73.971321
4,2,2018-01-01 00:14:09,2018-01-01 00:44:06,1,8.77,1,N,41,223,1,...,6.06,0.00,0.3,36.36,NaN,NaN,Manhattan,Central Harlem,40.776676,-73.971321


In [38]:
data_zone_withoutNaNLatLon.loc[0:5,["PULat","PULon","tpep_pickup_datetime"]]

,PULat,PULon,tpep_pickup_datetime
0,40.776676,-73.971321,2018-01-01 00:21:05
1,40.776676,-73.971321,2018-01-01 00:49:32
2,40.776676,-73.971321,2018-01-01 00:11:56
3,40.776676,-73.971321,2018-01-01 00:58:32
4,40.776676,-73.971321,2018-01-01 00:14:09
5,40.776676,-73.971321,2018-01-01 00:32:15


In [35]:
# weather=mgr.one_call_history(lat=data_zone_withoutNaNLatLon["PULat"],lon=data_zone_withoutNaNLatLon["PULon"],dt=data_zone_withoutNaNLatLon["tpep_pickup_datetime"].astype('int64')// 10**9)

data_zone_withoutNaNLatLon.loc[0:100,["PULat","PULon","tpep_pickup_datetime"]].apply(lambda x: mgr.one_call(lat=x["PULat"],lon=x["PULon"],dt=int(x["tpep_pickup_datetime"].timestamp())).current.status, axis=1)


0      Clouds
1      Clouds
2      Clouds
3      Clouds
4      Clouds
        ...  
96     Clouds
97     Clouds
98     Clouds
99     Clouds
100    Clouds
Length: 101, dtype: object

In [87]:
data_zone["tpep_pickup_datetime"].unique()
# .astype('int64')// 10**9

array(['2018-01-01T00:21:05.000000000', '2018-01-01T00:49:32.000000000',
       '2018-01-01T00:11:56.000000000', ...,
       '2018-01-07T04:41:08.000000000', '2018-01-06T03:50:28.000000000',
       '2018-01-15T05:44:01.000000000'], dtype='datetime64[ns]')

In [108]:
mgr = owm.weather_manager()

# what is the epoch for 3 days ago at this time?
# three_days_ago_epoch = int((datetime.now() - timedelta(days=3)).replace(tzinfo=timezone.utc).timestamp())

especifict_time = int(datetime(2020,12,8).replace(tzinfo=pytz.timezone("US/Manhattan")).timestamp())

one_call_especifict_time = mgr.one_call(lat=52.5244, lon=13.4105, dt=especifict_time)
one_call_especifict_time.current.detailed_status

UnknownTimeZoneError: 'US/Manhattan'

In [20]:
from datetime import datetime, timedelta

def date_range(start, end):
    delta = end - start  # as timedelta
    days = [start + timedelta(hours=i) for i in range(delta.days*24 + 1)]
    return days

start_date = datetime(2018, 1, 1)
end_date = datetime(2018, 1, 2,)
    
dates=date_range(start_date, end_date)
dates

[datetime.datetime(2018, 1, 1, 0, 0),
 datetime.datetime(2018, 1, 1, 1, 0),
 datetime.datetime(2018, 1, 1, 2, 0),
 datetime.datetime(2018, 1, 1, 3, 0),
 datetime.datetime(2018, 1, 1, 4, 0),
 datetime.datetime(2018, 1, 1, 5, 0),
 datetime.datetime(2018, 1, 1, 6, 0),
 datetime.datetime(2018, 1, 1, 7, 0),
 datetime.datetime(2018, 1, 1, 8, 0),
 datetime.datetime(2018, 1, 1, 9, 0),
 datetime.datetime(2018, 1, 1, 10, 0),
 datetime.datetime(2018, 1, 1, 11, 0),
 datetime.datetime(2018, 1, 1, 12, 0),
 datetime.datetime(2018, 1, 1, 13, 0),
 datetime.datetime(2018, 1, 1, 14, 0),
 datetime.datetime(2018, 1, 1, 15, 0),
 datetime.datetime(2018, 1, 1, 16, 0),
 datetime.datetime(2018, 1, 1, 17, 0),
 datetime.datetime(2018, 1, 1, 18, 0),
 datetime.datetime(2018, 1, 1, 19, 0),
 datetime.datetime(2018, 1, 1, 20, 0),
 datetime.datetime(2018, 1, 1, 21, 0),
 datetime.datetime(2018, 1, 1, 22, 0),
 datetime.datetime(2018, 1, 1, 23, 0),
 datetime.datetime(2018, 1, 2, 0, 0)]

In [28]:
owm = OWM(API_KEY)
mgr = owm.weather_manager()
mgr.one_call(lat=52.5244, lon=13.4105, dt=1543690560)



<pyowm.weatherapi25.one_call.OneCall - lat=52.5244, lon=13.4105, retrieval_time=1657217710>

In [14]:
owm = OWM(API_KEY)
mgr = owm.weather_manager()
mgr.one_call(lat=52.5244, lon=13.4105, dt=1543672560).current.to_dict()

{'reference_time': 1657216658,
 'sunset_time': 1657222179,
 'sunrise_time': 1657162346,
 'clouds': 75,
 'rain': {'1h': 1.32},
 'snow': {},
 'wind': {'speed': 4.92, 'deg': 268, 'gust': 6.71},
 'humidity': 89,
 'pressure': {'press': 1007, 'sea_level': None},
 'temperature': {'temp': 290.5, 'feels_like': 290.61},
 'status': 'Rain',
 'detailed_status': 'moderate rain',
 'weather_code': 501,
 'weather_icon_name': '10d',
 'visibility_distance': 10000,
 'dewpoint': 288.67,
 'humidex': None,
 'heat_index': None,
 'utc_offset': None,
 'uvi': 0.13,
 'precipitation_probability': None}

In [21]:
weathers_Manhattan=[]
for time in dates:

    especifict_time = int(time.replace(tzinfo=pytz.timezone("US/Eastern")).timestamp())
    # # (40.776676, -73.971321)
    # print(especifict_time)
    # print(mgr.one_call(lat=52.5244, lon=13.4105, dt=especifict_time).current.to_dict())
    one_call_especifict_time = mgr.one_call(lat=52.5244, lon=13.4105, dt=especifict_time)
    print(one_call_especifict_time.current.to_dict())
    # weathers_Manhattan.append(one_call_especifict_time.current.to_dict())

{'reference_time': 1657217315, 'sunset_time': 1657222179, 'sunrise_time': 1657162346, 'clouds': 75, 'rain': {'1h': 3.84}, 'snow': {}, 'wind': {'speed': 4.63, 'deg': 270}, 'humidity': 89, 'pressure': {'press': 1007, 'sea_level': None}, 'temperature': {'temp': 290.42, 'feels_like': 290.53}, 'status': 'Rain', 'detailed_status': 'light intensity shower rain', 'weather_code': 520, 'weather_icon_name': '09d', 'visibility_distance': 10000, 'dewpoint': 288.59, 'humidex': None, 'heat_index': None, 'utc_offset': None, 'uvi': 0.13, 'precipitation_probability': None}
{'reference_time': 1657217315, 'sunset_time': 1657222179, 'sunrise_time': 1657162346, 'clouds': 75, 'rain': {'1h': 3.84}, 'snow': {}, 'wind': {'speed': 4.63, 'deg': 270}, 'humidity': 89, 'pressure': {'press': 1007, 'sea_level': None}, 'temperature': {'temp': 290.42, 'feels_like': 290.53}, 'status': 'Rain', 'detailed_status': 'light intensity shower rain', 'weather_code': 520, 'weather_icon_name': '09d', 'visibility_distance': 10000, '

In [120]:
pd.Series(weathers_Manhattan).unique()

array(['broken clouds'], dtype=object)

In [51]:
for wM in weathers_Manhattan:
    print(wM["reference_time"])

1657214697
1657214697
1657214697
1657214697
1657214697
1657214697
1657214697
1657214697
1657214697
1657214697
1657214697
1657214697
1657214697
1657214697
1657214697
1657214697
1657214697
1657214697
1657214697
1657214697
1657214697
1657214697
1657214697
1657214697
1657214697
